dataset : https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

In [ ]:
from tools import find_file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_conf = pd.read_csv(find_file("time_series_covid_19_confirmed.csv"))
df_death = pd.read_csv(find_file("time_series_covid_19_deaths.csv"))
df_recovered = pd.read_csv(find_file("time_series_covid_19_recovered.csv"))

In [ ]:
df_conf = df_conf.sort_values(
    by=['Province/State','Country/Region'])
df_conf = df_conf.reset_index(drop=True)

df_death = df_death.sort_values(
    by=['Province/State','Country/Region'])
df_death = df_death.reset_index(drop=True)

df_recovered = df_recovered.sort_values(
    by=['Province/State','Country/Region'])
df_recovered = df_recovered.reset_index(drop=True)

In [ ]:
df_conf_melted = df_conf.melt(id_vars=['Province/State', 
                              'Country/Region', 'Lat', 'Long'], 
                              value_vars=dates_conf, 
                              var_name='Date',
                              value_name='Confirmed')

df_death_melted = df_death.melt(id_vars=['Province/State', 
                                'Country/Region', 'Lat', 'Long'],                                
                                value_vars=dates_death,
                                var_name='Date', 
                                value_name='Deaths')

df_combined = pd.concat([df_conf_melted, 
                         df_death_melted["Deaths"]], 
                         axis = 1, 
                         sort = False)


df_daily = df_combined.groupby(["Date","Country/Region"]).aggregate(
    {'Confirmed': 'sum', 'Deaths': 'sum'})

In [ ]:

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (12,7)

for country, df_country in df_daily.groupby(level=1):    
    dates = list(df_country.index.get_level_values('Date'))
    confirmed = list(df_country.Confirmed)
    deaths = list(df_country.Deaths)
    
    plt.bar(dates, confirmed, color='lightblue', 
        label="Nombre total de cas confirmés")
    plt.bar(dates, deaths, color='red',
        label="Nombre total de décès signalés")
    plt.xlabel("Dates")
    plt.ylabel("Nombre de personnes")
    plt.title(country)
    plt.legend()
    plt.show()

In [ ]:
def plot_pie(df, column, title):
    labels = list(df.index)
    sizes = list(df[column])
    explode = [0 for n in range(10)]
    
    with plt.style.context(
        { "axes.prop_cycle" : plt.cycler("color", 
        plt.cm.tab20c.colors)}):
        fig1, ax1 = plt.subplots()
        ax1.pie(sizes, 
                explode=explode, 
                labels=labels, 
                autopct='%1.2f%%', 
                shadow=False, 
                startangle=0)
        ax1.axis('equal')  # Un rapport d'aspect égal garantit que le diagramme 
                           # sera dessiné en cercle.
        plt.legend()
        plt.title(title)
    
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    plt.show()
    
top10_countries_confirmed = df_most_confirmed_recent_sorted.head(10)
plot_pie(top10_countries_confirmed, 
         "Confirmed", 
         "Top 10 Pays pour les cas confirmés")

In [ ]:
import folium
# obtenir la date la plus récente du DataFrame
most_recent_date = df_combined['Date'].max()
# obtenir toutes les données de cette date la plus récente
df_latest_day = df_combined[df_combined.Date==most_recent_date]
# créer une carte
folium_map = folium.Map(location=[40.738, -73.98], zoom_start=4)
color="#E37222"
scale = 5000
for place in df_latest_day.index:
    lat = df_latest_day.loc[place]["Lat"]
    lng = df_latest_day.loc[place]["Long"]    
    confirmed = df_latest_day.loc[place]["Confirmed"]
    deaths    = df_latest_day.loc[place]["Deaths"]
    # Cas Confirmés
    # dessiner un cercle sur la province/état/région/pays
    marker_confirmed = folium.CircleMarker(location=[lat, lng],
                                 radius=confirmed/scale,
                                 color="green",
                                 fill=True)
    marker_confirmed.add_to(folium_map)
    
    # Cas Décès
    # dessiner un cercle sur la province/état/région/pays
    radius = deaths/scale if deaths>0 else 0.000000001
    marker_deaths = folium.CircleMarker(location=[lat, lng],
                                 radius=radius,
                                 color="red",
                                 fill=True)
    marker_deaths.add_to(folium_map)
folium_map.save("Covid-19 Map.html")